In [127]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.layers import (Dense,Flatten,SimpleRNN,InputLayer,Conv1D,Bidirectional,GRU,LSTM,BatchNormalization,Dropout,Input, Embedding,TextVectorization,RepeatVector)
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.losses import SparseCategoricalCrossentropy

In [128]:
text_list=[]
with open(r'fra.txt', encoding='utf-8') as f:
    
    text_lines = f.readlines()
    for line in text_lines:
        cols=line.split('\t')
        text_list.append(cols)
df=pd.DataFrame(text_list,columns=['english','french','CC']).drop(columns=['CC'])
df.to_csv(r'dataset.csv',index=False)

In [129]:
df=pd.read_csv(r'dataset.csv')
df

,english,french
0,Go.,Va !
1,Go.,Marche.
2,Go.,En route !
3,Go.,Bouge !
4,Hi.,Salut !
...,...,...
227810,Death is something that we're often discourage...,La mort est une chose qu'on nous décourage sou...
227811,Since there are usually multiple websites on a...,Puisqu'il y a de multiples sites web sur chaqu...
227812,If someone who doesn't know your background sa...,Si quelqu'un qui ne connaît pas vos antécédent...
227813,It may be impossible to get a completely error...,Il est peut-être impossible d'obtenir un Corpu...


In [ ]:
ENGLISH_SEQUENCE_LENGTH=75
FRENCH_SEQUENCE_LENGTH=75
VOCAB_SIZE=20000
EMBEDDING_DIMS=200
ENGLISH_SEQUENCE_LENGTH,FRENCH_SEQUENCE_LENGTH,VOCAB_SIZE,EMBEDDING_DIMS

(75, 75, 20000, 200)

In [ ]:
english_vectorize_layer=TextVectorization(
    standardize='lower_and_strip_punctuation',
    max_tokens=VOCAB_SIZE,
    output_mode='int',
    output_sequence_length=ENGLISH_SEQUENCE_LENGTH
)

french_vectorize_layer=TextVectorization(
    standardize='lower_and_strip_punctuation',
    max_tokens=VOCAB_SIZE,
    output_mode='int',
    output_sequence_length=FRENCH_SEQUENCE_LENGTH
)

In [ ]:
""" df['french_input']=df['french'].apply(lambda text: "[start] "+text)
df['french_output']=df['french'].apply(lambda text: text+" [end]") """
df['french']=df['french'].apply(lambda text: "[start] "+text+" [end]")
df['french']

0                                        [start] Va ! [end]
1                                     [start] Marche. [end]
2                                  [start] En route ! [end]
3                                     [start] Bouge ! [end]
4                                     [start] Salut ! [end]
                                ...                        
227810    [start] La mort est une chose qu'on nous décou...
227811    [start] Puisqu'il y a de multiples sites web s...
227812    [start] Si quelqu'un qui ne connaît pas vos an...
227813    [start] Il est peut-être impossible d'obtenir ...
227814    [start] « Je suis allée boire avec un ami de m...
Name: french, Length: 227815, dtype: object

In [ ]:
english_vectorize_layer.adapt(df['english'])

In [ ]:
french_vectorize_layer.adapt(df['french'])

In [ ]:
english_data=np.array(english_vectorize_layer(df['english']))
french_data=np.array(french_vectorize_layer(df['french']))
french_data

array([[  2, 102,   3, ...,   0,   0,   0],
       [  2, 800,   3, ...,   0,   0,   0],
       [  2,  23, 621, ...,   0,   0,   0],
       ...,
       [  2,  43, 160, ...,   0,   0,   0],
       [  2,  14,  15, ...,   0,   0,   0],
       [  2, 227,   0, ...,   0,   0,   0]], dtype=int64)

In [ ]:
french_train.shape

(216424, 68)

In [ ]:
english_train,english_test,french_train,french_test=train_test_split(english_data,french_data,test_size=0.05)

hidden_units=4
# Encoder
encoder_inputs = Input(shape=(ENGLISH_SEQUENCE_LENGTH,))
encoder_embed = Embedding(VOCAB_SIZE, EMBEDDING_DIMS)(encoder_inputs)
encoder_lstm = LSTM(hidden_units, return_state=True,activation='leaky_relu')
encoder_outputs, state_h, state_c = encoder_lstm(encoder_embed)
encoder_states = [state_h, state_c]

# Decoder
decoder_inputs = Input(shape=(FRENCH_SEQUENCE_LENGTH-1,))
decoder_embed = Embedding(VOCAB_SIZE, EMBEDDING_DIMS)(decoder_inputs)
decoder_lstm = LSTM(hidden_units, return_sequences=True, return_state=True,activation='leaky_relu')
decoder_outputs, _, _ = decoder_lstm(decoder_embed, initial_state=encoder_states)
decoder_dense = Dense(VOCAB_SIZE, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.compile(optimizer='adam', loss=SparseCategoricalCrossentropy(), metrics=['accuracy'])

model.summary()

Model: "model_10"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_23 (InputLayer)          [(None, 75)]         0           []                               
                                                                                                  
 input_24 (InputLayer)          [(None, 74)]         0           []                               
                                                                                                  
 embedding_24 (Embedding)       (None, 75, 200)      4000000     ['input_23[0][0]']               
                                                                                                  
 embedding_25 (Embedding)       (None, 74, 200)      4000000     ['input_24[0][0]']               
                                                                                           

In [ ]:
history = model.fit(
    [english_train, french_train[:,:-1]],
    french_train[:,1:],
    batch_size=1024,
    epochs=10,
    validation_split=0.1
)

# Plot training and validation loss
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()

# Plot training and validation accuracy
plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()

plt.tight_layout()
plt.show()

Epoch 1/10


ResourceExhaustedError: Graph execution error:

Detected at node 'model_10/dense_12/Softmax' defined at (most recent call last):
    File "c:\Users\srika\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\Users\srika\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "C:\Users\srika\AppData\Roaming\Python\Python310\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "C:\Users\srika\AppData\Roaming\Python\Python310\site-packages\traitlets\config\application.py", line 1043, in launch_instance
      app.start()
    File "C:\Users\srika\AppData\Roaming\Python\Python310\site-packages\ipykernel\kernelapp.py", line 725, in start
      self.io_loop.start()
    File "C:\Users\srika\AppData\Roaming\Python\Python310\site-packages\tornado\platform\asyncio.py", line 215, in start
      self